In [101]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import torch.utils.data as Data
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [102]:
#hyperparameter
BatchSize = 128
lr = 0.001
EPOCH = 10

In [103]:
# def df_dataloader(df,BatchSize,is_shuffle=True):
#     df_target = torch.from_numpy(df['label'].values).float()
#     df_data = torch.from_numpy(((df.iloc[:,1:].values)/255.).reshape(-1,1,28,28)).float()
#     df = Data.TensorDataset(df_data,df_target)
#     df_dataloader = DataLoader(df,batch_size=BatchSize,shuffle=is_shuffle)
    
#     return df_dataloader

In [104]:
mean = (0.5088964127604166, 0.48739301317401956, 0.44194221124387256)
std = (0.2682515741720801, 0.2573637364478126, 0.2770957707973042)
transform_train = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
    #CIFAR100_train
CIFAR100_training = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
train_dataloader = DataLoader(CIFAR100_training, shuffle=True, num_workers=2, batch_size=BatchSize)

Files already downloaded and verified


In [105]:
transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    #CIFAR100_test = CIFAR100Test(path, transform=transform_test)
CIFAR100_test = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
test_dataloader = DataLoader(CIFAR100_test, shuffle=True, num_workers=2, batch_size=BatchSize)

Files already downloaded and verified


In [106]:
class ChannelAttention(nn.Module):
    def __init__(self,input_nc,rotio = 16):
        super().__init__()
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.maxpool = nn.AdaptiveMaxPool2d(1)
        self.share_mlp = nn.Sequential(nn.Conv2d(in_channels=input_nc,out_channels=input_nc//rotio,kernel_size=1,bias=False),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(in_channels=input_nc//rotio,out_channels=input_nc,kernel_size=1,bias=False))
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        avg_x = self.share_mlp(self.avgpool(x))
        max_x = self.share_mlp(self.maxpool(x))
        return self.sigmoid(avg_x+max_x)


class SpatialAttention(nn.Module):
    def __init__(self,kernel=7):
        super().__init__()
        assert kernel in (3,7),"Kernel size must be in 3 or 7"
        padding = 1 if kernel == 3 else 3
        self.cnn = nn.Conv2d(2,1,kernel_size=kernel,padding=padding,bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        avg_out = torch.mean(x,dim=1,keepdim=True)
        max_out = torch.max(x,dim=1,keepdim=True).values
        x = torch.cat([avg_out,max_out],dim=1)
        x = self.cnn(x)
        return self.sigmoid(x)
    
class cbam(nn.Module):
    def __init__(self,input_nc):
        super().__init__()
        self.ca = ChannelAttention(input_nc)
        self.sa = SpatialAttention()
        
    def forward(self,x):
        x = self.ca(x) * x
        x = self.sa(x) * x
        return x
    
    def get_spatial(self,x):
        return self.sa(x)

In [107]:
class Net(nn.Module):
    def __init__(self,is_cbam = True):
        super().__init__()
        self.is_cbam = is_cbam
        self.cnn1 = nn.Conv2d(3,128,3,1,1)
        self.relu1 = nn.ReLU(True)
        self.maxpool1 = nn.MaxPool2d(2)
        self.cnn2 = nn.Conv2d(128,64,3,1,1)
        self.relu2 = nn.ReLU(True)
        self.maxpool2 = nn.MaxPool2d(2)
        self.cbam = cbam(128)
        self.linear = nn.Linear(64*8*8,100)
    
    def forward(self,x):
        x = self.relu1(self.cnn1(x))
        x = self.maxpool1(x)
        if self.is_cbam:
            x = self.cbam(x)
            spatial_mask = self.cbam.get_spatial(x)
        x = self.relu2(self.cnn2(x))
        x = self.maxpool2(x)
        x = self.linear(x.view(-1,64*8*8))
        return x,spatial_mask

In [108]:
net = Net(is_cbam=True).to(device)
optimizer = optim.Adam(net.parameters(),lr = lr)
loss_fn = nn.CrossEntropyLoss()

In [109]:
def train_fn(model,optimizer,loader,loss_fn):
    model.train()
    for batch,(x,y) in enumerate(loader):
        x = x.to(device)
        y = y.to(device)
        pred, _ = model(x)
        loss = loss_fn(pred,y.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_pred = loss.item()
        
        if batch % 50 == 0:
            print(f"{batch*len(x)}/{len(loader.dataset)}   Loss:{loss_pred}")

def valid_fn(model,loader,loss_fn):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            valid_pred,_= model(x)
            test_loss += loss_fn(valid_pred,y.long()).item()
            correct += (valid_pred.argmax(1)).eq(y).sum()
    
    test_loss/= len(loader.dataset)
    print(f"\nTest Result: Average Loss = {test_loss}, Accuracy = {correct}/{len(loader.dataset)} ({(correct/len(loader.dataset))*100}%)")

In [110]:
for epoch in range(EPOCH):
    print(f'\nEpoch: {epoch+1}')
    train_fn(net,optimizer,train_dataloader,loss_fn)
    valid_fn(net,test_dataloader,loss_fn)


Epoch: 1
0/50000   Loss:4.608028888702393
6400/50000   Loss:4.097858905792236
12800/50000   Loss:3.9647376537323
19200/50000   Loss:3.7224013805389404
25600/50000   Loss:3.632485866546631
32000/50000   Loss:3.538907766342163
38400/50000   Loss:3.6181042194366455
44800/50000   Loss:3.4915354251861572

Test Result: Average Loss = 0.026116641783714294, Accuracy = 2169/10000 (21.689998626708984%)

Epoch: 2
0/50000   Loss:3.233112335205078
6400/50000   Loss:3.1926512718200684
12800/50000   Loss:3.282860040664673
19200/50000   Loss:3.2825417518615723
25600/50000   Loss:3.138991594314575
32000/50000   Loss:3.278108596801758
38400/50000   Loss:3.2202858924865723
44800/50000   Loss:2.9068679809570312

Test Result: Average Loss = 0.022848853302001954, Accuracy = 3006/10000 (30.059999465942383%)

Epoch: 3
0/50000   Loss:3.0483503341674805
6400/50000   Loss:3.1004247665405273
12800/50000   Loss:3.0235612392425537
19200/50000   Loss:2.9170591831207275
25600/50000   Loss:2.8846185207366943
32000/50